In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('WeatherAnalysis').getOrCreate()

In [9]:
df=spark.read.csv('Cleaned_Weather_Data.csv',inferSchema=True,header=True)

In [10]:
df.createOrReplaceTempView('Aus_Weather_Data')

In [20]:
import time
start_time = time.time()
avg_temp = spark.sql('''
    select 
        Location,
        Round(avg(MinTemp_in_Celcius),2) AS AvgMinTemp,
        Round(avg(MaxTemp_in_Celcius),2) AS AvgMaxTemp
    from 
        Aus_Weather_Data
    group by 
        Location
''')
avg_temp.show()
end_time = time.time()
processing_time = end_time - start_time
print(f"Processing Time: {processing_time:.2f} seconds")
avg_temp.write.csv('Goal1',header=True)

+-------------+----------+----------+
|     Location|AvgMinTemp|AvgMaxTemp|
+-------------+----------+----------+
|       Cairns|     21.25|     29.57|
|NorfolkIsland|     16.92|     21.84|
|      Bendigo|      8.74|     22.04|
|      Walpole|     11.91|     20.56|
|     Canberra|      6.91|     20.94|
|      Woomera|     13.44|      26.7|
|     Adelaide|     12.87|     23.34|
|        Cobar|     12.82|     25.46|
|SydneyAirport|     14.98|     23.44|
| PerthAirport|     12.69|     25.84|
|   Wollongong|     15.22|     21.64|
|  Williamtown|     12.74|      24.1|
|        Moree|     12.63|     26.81|
|      Mildura|     10.76|     24.98|
|     Portland|      9.36|     17.81|
|       Albany|     13.05|     20.22|
|   SalmonGums|      9.27|     24.06|
|     Brisbane|     16.49|     26.51|
|       Sydney|     14.92|     23.05|
|        Perth|     12.99|     25.09|
+-------------+----------+----------+
only showing top 20 rows

Processing Time: 1.26 seconds


In [21]:
start_time = time.time()
extreme_temps = spark.sql('''
    select 
        Location, 
        min(MinTemp_in_Celcius) as Lowest_Temperature, 
        max(MaxTemp_in_Celcius) as Highest_Temperature
    from 
        Aus_Weather_Data
    where 
        MinTemp_in_Celcius < 0 OR MaxTemp_in_Celcius > 40
        group by 
        Location
''')
extreme_temps.show()
end_time = time.time()
processing_time = end_time - start_time
print(f"Processing Time: {processing_time:.2f} seconds")
extreme_temps.write.csv('Goal2',header=True)

+----------------+------------------+-------------------+
|        Location|Lowest_Temperature|Highest_Temperature|
+----------------+------------------+-------------------+
|         Bendigo|              -3.9|               45.4|
|        Canberra|              -7.6|               42.0|
|         Woomera|              18.6|               46.8|
|        Adelaide|              17.3|               45.7|
|           Cobar|              -2.0|               45.4|
|   SydneyAirport|              22.7|               46.4|
|    PerthAirport|              -1.0|               43.0|
|     Williamtown|              -1.4|               42.9|
|           Moree|              -1.6|               43.1|
|         Mildura|              -3.1|               46.0|
|        Portland|              15.0|               40.4|
|      SalmonGums|              -4.4|               46.3|
|          Sydney|              21.4|               40.9|
|           Perth|              20.9|               42.9|
|            S

In [22]:
import time
start_time = time.time()
toplocations = spark.sql('''
    select 
        Location, 
        round(avg(MaxTemp_in_Celcius),2) AS AvgMaxTemp 
    from 
        Aus_Weather_Data
    group by 
        Location
    order by 
        AvgMaxTemp DESC
    LIMIT 5
''')
toplocations.show()
end_time = time.time()
processing_time = end_time - start_time
print(f"Processing Time: {processing_time:.2f} seconds")
toplocations.write.csv('Goal3',header=True)

+----------+----------+
|  Location|AvgMaxTemp|
+----------+----------+
| Katherine|     34.97|
|    Darwin|     32.61|
|     Uluru|     30.65|
|    Cairns|     29.57|
|Townsville|     29.31|
+----------+----------+

Processing Time: 1.29 seconds


In [23]:
start_time = time.time()
sudden_drops = spark.sql('''
    WITH TempChanges AS (
        select 
            index,
            Location,
            MinTemp_in_Celcius,
            MaxTemp_in_Celcius,
            LAG(MinTemp_in_Celcius) OVER (PARTITION BY Location ORDER BY index) AS PrevMinTemp,
            LAG(MaxTemp_in_Celcius) OVER (PARTITION BY Location ORDER BY index) AS PrevMaxTemp
        from 
            Aus_Weather_Data
    )
    select 
        index,
        Location,
        MinTemp_in_Celcius,
        MaxTemp_in_Celcius,
        (PrevMinTemp - MinTemp_in_Celcius) AS MinTempDrop,
        (PrevMaxTemp - MaxTemp_in_Celcius) AS MaxTempDrop
    from 
        TempChanges
    where 
        (PrevMinTemp - MinTemp_in_Celcius) > 10 OR 
        (PrevMaxTemp - MaxTemp_in_Celcius) > 10
''')
sudden_drops.show()
end_time = time.time()
processing_time = end_time - start_time
print(f"Processing Time: {processing_time:.2f} seconds")
sudden_drops.write.csv('Goal4',header=True)

+-----+--------+------------------+------------------+-------------------+------------------+
|index|Location|MinTemp_in_Celcius|MaxTemp_in_Celcius|        MinTempDrop|       MaxTempDrop|
+-----+--------+------------------+------------------+-------------------+------------------+
|28871|Adelaide|              15.1|              25.2|-0.5999999999999996|10.000000000000004|
|28880|Adelaide|              17.2|              30.0|                8.7|10.600000000000001|
|28883|Adelaide|              16.3|              24.9|  5.899999999999999|              10.5|
|28886|Adelaide|              15.2|              24.0| 11.100000000000001| 8.200000000000003|
|28893|Adelaide|              15.0|              22.1| 1.3999999999999986|12.899999999999999|
|28899|Adelaide|              16.4|              21.7|  4.600000000000001|              12.2|
|28912|Adelaide|               9.9|              16.7|               11.4|               5.5|
|28964|Adelaide|              11.6|              16.5|  4.30

In [24]:
start_time = time.time()
humidity_analysis = spark.sql('''
    WITH TempCategory AS (
        select 
            *,
            case
                when MaxTemp_in_Celcius >= 30 THEN 'Hot'
                when MaxTemp_in_Celcius >= 20 AND MaxTemp_in_Celcius < 30 THEN 'Moderate'
                else 'Cool'
            end AS TempRange
        from 
            Aus_Weather_Data
    )
    select 
        TempRange,
        round(avg(humidity_at_9am),2) AS AvgHumidity9am,
        round(avg(humidity_at_3pm),2) AS AvgHumidity3pm,
        round(avg(Rainfall_in_millimeters),2) AS AvgRainfall
    from 
        TempCategory
    GROUP BY 
        TempRange
''')


humidity_analysis.show()
end_time = time.time()
processing_time = end_time - start_time
print(f"Processing Time: {processing_time:.2f} seconds")
humidity_analysis.write.csv('Goal5',header=True)

+---------+--------------+--------------+-----------+
|TempRange|AvgHumidity9am|AvgHumidity3pm|AvgRainfall|
+---------+--------------+--------------+-----------+
|     Cool|         78.27|         61.23|       2.96|
|      Hot|         54.34|         35.01|       1.39|
| Moderate|         67.48|         50.89|       2.26|
+---------+--------------+--------------+-----------+

Processing Time: 1.10 seconds


In [25]:
start_time = time.time()
rainy_days = spark.sql('''
    select 
        Location, 
        COUNT(*) AS RainyDays
    from 
        Aus_Weather_Data
    where 
        Rainfall_in_millimeters > 0
    GROUP BY 
        Location
''')
rainy_days.show()
end_time = time.time()
processing_time = end_time - start_time
print(f"Processing Time: {processing_time:.2f} seconds")
rainy_days.write.csv('Goal6',header=True)

+-------------+---------+
|     Location|RainyDays|
+-------------+---------+
|       Cairns|      396|
|NorfolkIsland|      492|
|      Bendigo|      323|
|      Walpole|      398|
|     Canberra|      304|
|      Woomera|      119|
|     Adelaide|      289|
|        Cobar|      182|
|SydneyAirport|      316|
| PerthAirport|      189|
|   Wollongong|      319|
|  Williamtown|      240|
|        Moree|      174|
|      Mildura|      169|
|     Portland|      528|
|       Albany|      415|
|   SalmonGums|      287|
|     Brisbane|      339|
|       Sydney|      379|
|        Perth|      243|
+-------------+---------+
only showing top 20 rows

Processing Time: 1.15 seconds
